<a href="https://colab.research.google.com/github/Pankajkashyap1/Hindi_2_Chhattisgarhi_Translation/blob/master/h2cg_using_LSTM_Done.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import the important Directories**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


In [ ]:
import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense,TimeDistributed,Embedding,Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from string import digits
import nltk
import re
import string
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1)

**Upload the DATASET from the local folder**

In [ ]:
from google.colab import files
uploaded= files.upload()

Saving work.xlsx to work (2).xlsx


In [ ]:
lines=pd.read_excel("work.xlsx")
lines = lines[:256]
lines.head()

,cg_sentance,hindi_sentance
0,नहाय के बाद सरीर ल बने अंगोछना चाही,नहाने के बाद शरीर को अच्छे से पोंछना चाहिए
1,अंगरा के सेके रोटी ल अंगाकर कथे,अंगर से सेके रोटी को चावल की मोती रोटी कहते हैं
2,मोर दाई ह अँचरा म पइसा गठियाय हे,मेरी मां ने सिक्कों को अपने आँचल में बांध रखा है
3,अंचावन ल अनदेखी करबे त खटिया ढिल्ला हो जाही,खाट की रस्सियों को कसने की रस्सी को नज़रअंदाज़ करेंगे तो खाट ढीली हो जाएगी
4,दिन के अंजोर म रद्दा नइ भूला हूँ,दिन के उजाले में रास्ते से नहीं भटकूंगा


**Preprocessing steps**

In [ ]:
# Lowercase all characters
lines['hindi_sentance']=lines['hindi_sentance'].apply(lambda x: str(x))
lines['cg_sentance']=lines['cg_sentance'].apply(lambda x: str(x))
lines['hindi_sentance']=lines['hindi_sentance'].apply(lambda x: x.lower())
lines['cg_sentance']=lines['cg_sentance'].apply(lambda x: x.lower())

In [ ]:
# Remove quotes
lines['hindi_sentance']=lines['hindi_sentance'].apply(lambda x: re.sub("'", '', x))
lines['cg_sentance']=lines['cg_sentance'].apply(lambda x: re.sub("'", '', x))

In [ ]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines['hindi_sentance']=lines['hindi_sentance'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['cg_sentance']=lines['cg_sentance'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [ ]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines['hindi_sentance']=lines['hindi_sentance'].apply(lambda x: x.translate(remove_digits))
lines['cg_sentance']=lines['cg_sentance'].apply(lambda x: x.translate(remove_digits))

lines['cg_sentance'] = lines['cg_sentance'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
lines['hindi_sentance']=lines['hindi_sentance'].apply(lambda x: x.strip())
lines['cg_sentance']=lines['cg_sentance'].apply(lambda x: x.strip())
lines['hindi_sentance']=lines['hindi_sentance'].apply(lambda x: re.sub(" +", " ", x))
lines['cg_sentance']=lines['cg_sentance'].apply(lambda x: re.sub(" +", " ", x))

In [ ]:
# Add start and end tokens to target sequences
lines['cg_sentance'] = lines['cg_sentance'].apply(lambda x : 'START_ '+ x + ' _END')

In [ ]:
### Get Hindi and CG Vocabulary
all_hindi_words=set()
for hindi in lines['hindi_sentance']:
    for word in hindi.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

all_cg_words=set()
for cg in lines['cg_sentance']:
    for word in cg.split():
        if word not in all_cg_words:
            all_cg_words.add(word)

In [ ]:
lines['length_hindi_sentance']=lines['hindi_sentance'].apply(lambda x:len(x.split(" ")))
lines['length_cg_sentance']=lines['cg_sentance'].apply(lambda x:len(x.split(" ")))

In [ ]:
lines.head()
lines[lines['length_hindi_sentance']>30].shape

(0, 4)

In [ ]:
lines=lines[lines['length_hindi_sentance']<=20]
lines=lines[lines['length_cg_sentance']<=20]

In [ ]:
print("maximum length of CG Sentence ",max(lines['length_cg_sentance']))
print("maximum length of HINDI Sentence ",max(lines['length_hindi_sentance']))

maximum length of CG Sentence  16
maximum length of HINDI Sentence  16


In [ ]:
max_length_src=max(lines['length_cg_sentance'])
max_length_tar=max(lines['length_hindi_sentance'])

**Generate tokens for hindi words and  CG words**

In [ ]:
input_words = sorted(list(all_hindi_words))
target_words = sorted(list(all_cg_words))
num_encoder_tokens = len(all_hindi_words)
num_encoder_tokens = num_encoder_tokens +1
num_decoder_tokens = len(all_cg_words)
num_encoder_tokens, num_decoder_tokens

(750, 793)

In [ ]:
num_decoder_tokens += 1

In [ ]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [ ]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [ ]:
lines.head(10)

,cg_sentance,hindi_sentance,length_hindi_sentance,length_cg_sentance
0,START_ नहाय के बाद सरीर ल बने अंगोछना चाही _END,नहाने के बाद शरीर को अच्छे से पोंछना चाहिए,9,10
1,START_ अंगरा के सेके रोटी ल अंगाकर कथे _END,अंगर से सेके रोटी को चावल की मोती रोटी कहते हैं,11,9
2,START_ मोर दाई ह अँचरा म पइसा गठियाय हे _END,मेरी मां ने सिक्कों को अपने आँचल में बांध रखा है,11,10
3,START_ अंचावन ल अनदेखी करबे त खटिया ढिल्ला हो जाही _END,खाट की रस्सियों को कसने की रस्सी को नज़रअंदाज़ करेंगे तो खाट ढीली हो जाएगी,15,11
4,START_ दिन के अंजोर म रद्दा नइ भूला हूँ _END,दिन के उजाले में रास्ते से नहीं भटकूंगा,8,10
5,START_ बने अंजोर म पढ़ना लिखना चाही _END,उचित प्रकाश में पढ़ना लिखना करना चाहिए,7,8
6,START_ हमन ल अंजोरी रात म चले म मजा आइस _END,हमें चांदनी रात में घूमने में मज़ा आया,8,11
7,START_ अण्डी तेल ह मालिस बर अच्छा होथे _END,अरंडी का तेल मालिश के लिए अच्छा होता है,9,9
8,START_ अर्जुन ह पांडव मन म अंतरमंझिला रिहिस _END,पांडवों में अर्जुन पाँच भाइयों में तीसरे थे,8,9
9,START_ गणित म अंताजी नइ चलय _END,गणित में अनुमान नही चलता,5,7


**Create train & test model **

In [ ]:
from sklearn.model_selection import train_test_split
X, y = lines['hindi_sentance'], lines['cg_sentance']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((204,), (52,))

In [ ]:
X_train

143    तुमने लिखा                                                                
84     हरिया का घर यहीं से सीधा है कहीं और मत जाना                               
55     मुसीबत में काम आये वही मित्र है                                           
220    अलवा जलवा हलवा भी स्वादिष्ठ लगता है                                       
104    मेरे छोटे बेटे का हाथ पंखे के बटन तक नहीं पहुँचता                         
139    वे दो घंटे से रामायण पढ़ रहे हैं                                          
127    तुम खा रहे हो                                                             
60     जिद्दी लड़का किसी की नहीं सुनता                                           
101    यह किताब उस बहन तक पहुँचा देना                                            
172    सीता पढ़ेगी                                                               
245    फूल मुरझा गया है                                                          
126    हम खा रहे हैं                                                             
225    जो गाली द

In [ ]:
len(X_test)

52

**Creation of LSTM model**

Long short-term memory (LSTM) is a type of recurrent neural network (RNN) aimed at dealing with the vanishing gradient problem present in traditional RNNs.

**Structure of LSTM**

An LSTM (Long Short-Term Memory) network is a type of RNN recurrent neural network that is capable of handling and processing sequential data. The structure of an LSTM network consists of a series of LSTM cells, each of which has a set of gates (input, output, and forget gates) that control the flow of information into and out of the cell. The gates are used to selectively forget or retain information from the previous time steps, allowing the LSTM to maintain long-term dependencies in the input data.

LSTM_Recurrent_unit.avif

In [ ]:
def generate_batch(X = X_train, y = y_train, batch_size = 32):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [ ]:
latent_dim = 200
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [ ]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 32
epochs = 100

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 200)            150000    ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, None, 200)            158800    ['input_2[0][0]']             
                                                                                              

In [ ]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=100,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

Epoch 1/100


<ipython-input-29-49141b86a382>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),


6/6 [==============================] - 18s 793ms/step - loss: 6.6691 - accuracy: 0.1077 - val_loss: 6.6537 - val_accuracy: 0.1322
Epoch 2/100
6/6 [==============================] - 1s 152ms/step - loss: 6.5951 - accuracy: 0.1469 - val_loss: 6.4471 - val_accuracy: 0.1258
Epoch 3/100
6/6 [==============================] - 1s 135ms/step - loss: 6.0915 - accuracy: 0.1468 - val_loss: 6.1048 - val_accuracy: 0.1322
Epoch 4/100
6/6 [==============================] - 1s 160ms/step - loss: 5.6325 - accuracy: 0.1455 - val_loss: 5.9756 - val_accuracy: 0.1258
Epoch 5/100
6/6 [==============================] - 1s 155ms/step - loss: 5.4924 - accuracy: 0.1490 - val_loss: 6.3560 - val_accuracy: 0.1405
Epoch 6/100
6/6 [==============================] - 1s 137ms/step - loss: 5.3963 - accuracy: 0.1568 - val_loss: 6.2399 - val_accuracy: 0.1384
Epoch 7/100
6/6 [==============================] - 1s 139ms/step - loss: 5.2777 - accuracy: 0.1559 - val_loss: 6.5957 - val_accuracy: 0.1488
Epoch 8/100
6/6 [=======

In [ ]:
model.save('hindi_to_cg.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Hindi sentence:', X_train[k:k+1].values[0])
print('Actual cg Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted cg Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 26ms/step
Input Hindi sentence: तुमने लिखा
Actual cg Translation:  तुमन लिखेव 
Predicted cg Translation:  तें लिखेस 


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Hindi sentence:', X_train[k:k+1].values[0])
print('Actual cg Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted cg Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 24ms/step
Input Hindi sentence: हरिया का घर यहीं से सीधा है कहीं और मत जाना
Actual cg Translation:  हरिया के घर ईंहा ले सोज्झे हे अंते झन जाबे 
Predicted cg Translation:  हरिया के घर ईंहा ले सोज्झे हे अंते झन जाबे 


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Hindi sentence:', X_train[k:k+1].values[0])
print('Actual cg Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted cg Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 23ms/step
Input Hindi sentence: मुसीबत में काम आये वही मित्र है
Actual cg Translation:  अटके परे म काम आथे तउने मितान हे 
Predicted cg Translation:  अटके परे म काम आथे तउने मितान हे 


**Take a input from User**

In [39]:
from keras.models import load_model, Model
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Load the saved model
model = load_model('hindi_to_cg.h5')

# Assume you have already defined and loaded `encoder_model` and `decoder_model` as per the previous steps.

# Function to process input sentence
def process_input(input_text, input_token_index, max_length_src):
    input_seq = np.zeros((1, max_length_src))
    for t, word in enumerate(input_text.split()):
        if word in input_token_index:
            input_seq[0, t] = input_token_index[word]
        else:
            input_seq[0, t] = input_token_index['<UNK>']  # Handle unknown words if needed
    return input_seq

# Function to decode sequence using decoder model
def decode_sequence(input_seq, encoder_model, decoder_model, target_token_index, reverse_target_char_index):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' ' + sampled_char

        # Exit condition: either hit max length or find stop character.
        if (sampled_char == '_END' or len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

# Example usage to process input from user
def main():
    while True:
        input_text = input("Enter a Hindi sentence (type 'exit' to quit): ").strip().lower()
        if input_text == 'exit':
            break

        # Process input
        input_seq = process_input(input_text, input_token_index, max_length_src)

        # Decode sequence
        decoded_sentence = decode_sequence(input_seq, encoder_model, decoder_model, target_token_index, reverse_target_char_index)

        # Print results
        print('Input Hindi sentence:', input_text)
        print('Predicted cg Translation:', decoded_sentence[:-4])  # Remove _END token from output

# Assuming you have `input_token_index`, `target_token_index`, and `reverse_target_char_index` defined already

# Example call to start the application
if __name__ == '__main__':
    main()


Enter a Hindi sentence (type 'exit' to quit): यह प्रश्न साधारण है  
1/1 [==============================] - 0s 24ms/step
Input Hindi sentence: यह प्रश्न साधारण है
Predicted cg Translation:  ए सवाल सोज्झे हे 
Enter a Hindi sentence (type 'exit' to quit): गणित में तीर तुक्का नहीं चलता 
1/1 [==============================] - 0s 22ms/step
Input Hindi sentence: गणित में तीर तुक्का नहीं चलता
Predicted cg Translation:  गणित में अटकरपंचे नी चलय 
Enter a Hindi sentence (type 'exit' to quit): exit


**Bleu Score**

In [76]:
import numpy as np
from keras.models import load_model, Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import pandas as pd
import re

In [77]:
# Function to process input sentence
def process_input(input_text, input_token_index, max_length_src):
    input_seq = np.zeros((1, max_length_src))
    for t, word in enumerate(input_text.split()):
        if word in input_token_index:
            input_seq[0, t] = input_token_index[word]
        else:
            input_seq[0, t] = input_token_index['<UNK>']  # Handle unknown words
    return input_seq


In [78]:
# Function to decode sequence using decoder model
def decode_sequence(input_seq, encoder_model, decoder_model, target_token_index, reverse_target_char_index):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' ' + sampled_char

        # Exit condition: either hit max length or find stop character.
        if (sampled_char == '_END' or len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    # Remove start and end tokens
    decoded_sentence = decoded_sentence.replace('START_', '').replace('_END', '').strip()
    return decoded_sentence


In [72]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [79]:
# Function to calculate BLEU score with smoothing, ignoring '<UNK>' token
def calculate_bleu(actual, predicted):
    chencherry = SmoothingFunction()
    actual_tokens = [token for token in nltk.word_tokenize(actual) if token != '<UNK>']
    predicted_tokens = [token for token in nltk.word_tokenize(predicted) if token != '<UNK>']
    return sentence_bleu([actual_tokens], predicted_tokens, smoothing_function=chencherry.method1)


In [80]:
# Function to calculate average BLEU score for dataset
def calculate_average_bleu_for_dataset(dataset, input_token_index, max_length_src, encoder_model, decoder_model, target_token_index, reverse_target_char_index):
    total_bleu_score = 0.0
    num_samples = len(dataset)

    for index, row in dataset.iterrows():
        input_text = row['hindi_sentance']
        actual_output = row['cg_sentance']

        # Process input
        input_seq = process_input(input_text, input_token_index, max_length_src)

        # Decode sequence
        predicted_output = decode_sequence(input_seq, encoder_model, decoder_model, target_token_index, reverse_target_char_index)

        # Calculate BLEU score
        bleu_score = calculate_bleu(actual_output, predicted_output)
        total_bleu_score += bleu_score

    average_bleu_score = total_bleu_score / num_samples
    return average_bleu_score


In [ ]:
# Load your dataset
lines = pd.read_excel("work.xlsx")  # Adjust the file path as needed
lines = lines[:256]

# Preprocess the dataset (assuming the preprocessing steps are the same as before)
lines['hindi_sentance'] = lines['hindi_sentance'].apply(lambda x: str(x).lower().strip())
lines['cg_sentance'] = lines['cg_sentance'].apply(lambda x: 'START_ ' + str(x).lower().strip() + ' _END')
lines['hindi_sentance'] = lines['hindi_sentance'].apply(lambda x: re.sub(" +", " ", x))
lines['cg_sentance'] = lines['cg_sentance'].apply(lambda x: re.sub(" +", " ", x))

# Assume `input_token_index`, `target_token_index`, `reverse_target_char_index`, `max_length_src`, `encoder_model`, and `decoder_model` are defined as before
# Example call to calculate average BLEU score for the dataset
average_bleu_score = calculate_average_bleu_for_dataset(lines, input_token_index, max_length_src, encoder_model, decoder_model, target_token_index, reverse_target_char_index)

# Print average BLEU score
print(f'Average BLEU Score for the dataset: {average_bleu_score:.4f}')